<a href="https://colab.research.google.com/github/tahreemrasul/gemini_workshops/blob/main/react_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a react agent using Gemini

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/tahreemrasul/gemini_workshops/blob/main/react_agent.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/tahreemrasul/gemini_workshops/blob/main/react_agent.ipynb">
      <img width="32px" src="https://raw.githubusercontent.com/primer/octicons/refs/heads/main/icons/mark-github-24.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>  

| Author |
| --- |
| [Tahreem Rasul](https://github.com/tahreemrasul/) |

### What is a ReAct Agent?

**ReAct** = Reasoning + Acting.

The agent follows this loop:

> **Thought → (optional) Tool Action → Observation → Thought → … → Final Answer**

Concretely:

1. The model **thinks** about the question.
2. It decides whether it needs a **tool** (e.g. weather, calculator, knowledge).
3. If yes:
   - we call the tool in Python (this is the **Action**),
   - capture the result (the **Observation**).
4. We feed the observation back into the model,
5. It continues reasoning,
6. Eventually it stops calling tools and returns a **final answer**.

---

### What We'll Build

We'll expose three tools:

1. `get_weather(city)` - mock weather API
2. `calculate(expression)` - tiny calculator (demo only)
3. `get_country_info(country)` - small knowledge base for countries

Our ReAct agent will:

- read the user's question,
- use one or more tools as needed,
- combine tool results with its own reasoning,
- return a final answer.


## Getting Started

### Install & Import Dependencies

In [ ]:
!pip install -U -q google-genai

In [ ]:
import os
import getpass
from typing import Dict

from google import genai
from google.genai import types

print("google-genai version:", genai.__version__)

google-genai version: 1.54.0


### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Get API Key

There are a few different ways of accessing the Gemini models in your applications. Most notably, you can do this through Vertex AI in Google Cloud Platform, or via the API method using Google AI Studio. We will be using the second approach.

Head over to https://aistudio.google.com/ and generate an API Key.
1. Click on **API Keys** on the left sidebar.
2. You should now see an option to **Create API Key**. Click on this.
3. A dialog box will open. You will be prompted to enter the name of the key. Enter a name of your choice. You also need to link this to a Google Cloud Project. From the dropdown of project selection, select **Create project**. You will be prompted to name the project, choose a name of your choice. Click on **Create Key**.
4. You should now see your created API key in the list. Click on the key and copy it. Paste it in the cell below.


In [ ]:
# 🔑 Paste your API key from Google AI Studio
API_KEY = "your-api-key"

from google import genai
client = genai.Client(api_key=API_KEY)

### Choose a model

In [ ]:
MODEL_ID = "gemini-2.5-flash"  # @param {type: "string"}

## Build the agent

### Step 1 - Defining Tools (Functions) for the Agent

We'll create **three tools**, each as a normal Python function plus a
structured **FunctionDeclaration** so Gemini can call it via function calling:

1. `get_weather(city)`  
   - Mock weather info for a few cities.

2. `calculate(expression)`  
   - Tiny calculator using Python expressions (for demo only).

3. `get_country_info(country)`  
   - Mini knowledge base with country facts (capital, region, fun fact).

We then bundle all three into a single `Tool` object, which we will pass
to the model on each step.


In [ ]:
from typing import Any, Dict

# 1) Mock weather tool
def get_weather_impl(city: str) -> Dict[str, Any]:
    """Mock weather function.

    In a real system, this would call a weather API.
    Here we just return fake but structured data.
    """
    fake_temps = {
        "berlin": {"temperature_c": 17, "condition": "cloudy"},
        "hamburg": {"temperature_c": 15, "condition": "windy"},
        "lahore": {"temperature_c": 32, "condition": "hot"},
        "islamabad": {"temperature_c": 28, "condition": "warm and clear"},
        "london": {"temperature_c": 19, "condition": "light rain"},
    }

    city_key = city.lower()
    data = fake_temps.get(
        city_key,
        {"temperature_c": 22, "condition": "mild and unknown"},
    )
    return {
        "city": city,
        "temperature_c": data["temperature_c"],
        "condition": data["condition"],
    }

In [ ]:
# 2) Tiny calculator tool  (demo only)
def calculate_impl(expression: str) -> Dict[str, Any]:
    """Very simple calculator for demo purposes.

    Evaluates a basic Python arithmetic expression.
    Do NOT use this pattern in production.
    """
    try:
        result = eval(expression, {"__builtins__": {}}, {})
        return {"ok": True, "result": result}
    except Exception as e:
        return {"ok": False, "error": str(e)}

In [ ]:
# 3) Country info / mini-knowledge-base tool
def get_country_info_impl(country: str) -> Dict[str, Any]:
    """Return some static info about a few countries."""
    country_db = {
        "germany": {
            "capital": "Berlin",
            "region": "Europe",
            "fun_fact": "Known for its engineering, classical music, and strong research culture.",
        },
        "pakistan": {
            "capital": "Islamabad",
            "region": "South Asia",
            "fun_fact": "Home to some of the world's highest mountain peaks, including K2.",
        },
        "turkiye": {
            "capital": "Ankara",
            "region": "Europe / Asia",
            "fun_fact": "Istanbul is the only major city that spans two continents.",
        },
        "united kingdom": {
            "capital": "London",
            "region": "Europe",
            "fun_fact": "London has an underground mail rail that once moved letters beneath the city.",
        },
    }

    key = country.lower().strip()
    info = country_db.get(
        key,
        {
            "capital": "Unknown",
            "region": "Unknown",
            "fun_fact": "No data available in this tiny demo knowledge base.",
        },
    )
    return {"country": country, **info}

We will now describe the functions to the model.

In [ ]:
# Function declarations tell the model how to call these tools.

get_weather_decl = types.FunctionDeclaration(
    name="get_weather",
    description="Get the current (mock) weather for a given city.",
    parameters={
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "City name, e.g. 'London' or 'Hamburg'.",
            }
        },
        "required": ["city"],
    },
)

calculate_decl = types.FunctionDeclaration(
    name="calculate",
    description="Evaluate a simple math expression, like '2 + 3 * 4'.",
    parameters={
        "type": "object",
        "properties": {
            "expression": {
                "type": "string",
                "description": "Math expression in Python syntax, e.g. '2 + 3 * 4'.",
            }
        },
        "required": ["expression"],
    },
)

get_country_info_decl = types.FunctionDeclaration(
    name="get_country_info",
    description="Get basic information about a country (capital, region, fun fact).",
    parameters={
        "type": "object",
        "properties": {
            "country": {
                "type": "string",
                "description": "Country name, e.g. 'Pakistan' or 'Germany'.",
            }
        },
        "required": ["country"],
    },
)

# Bundle all three declarations into a single Tool.
react_tool = types.Tool(
    function_declarations=[
        get_weather_decl,
        calculate_decl,
        get_country_info_decl,
    ]
)

print("Tools defined:", [fd.name for fd in react_tool.function_declarations])

Tools defined: ['get_weather', 'calculate', 'get_country_info']


### Step 2 - Create ReAct system instruction

Next up, we will define the system prompt for our ReAct agent. This tells the model to:

- Think step-by-step.
- Use tools when they are useful.
- Use multiple tools if needed.
- Stop calling tools once it can give a final answer.

We will keep a separate `scratchpad` where the agent's “Thoughts” and
“Observations” accumulate. That scratchpad will be included in the
context on each step, but not shown to the end user.

In [ ]:
REACT_SYSTEM_PROMPT = """
You are a ReAct-style agent.

Rules:
- Think privately (do NOT output your chain-of-thought).
- If you need external information or a calculation, call one of the tools.
- After receiving a tool result, use it to update your reasoning internally.
- When you have enough information, output ONLY the final answer to the user.
- Do NOT prefix answers with "Thought", "Step", or anything else.
- If you call a tool, do NOT output the final answer in the same turn.

Tools available:
- get_weather(city)
- calculate(expression)
- get_country_info(country)
"""

### Step 3 - One ReAct Step

We will first define a helper function that executes **one step** of the ReAct loop:

- Input:
  - the original `question`,
  - the current `scratchpad` (Thoughts + Observations so far),
  - the last tool response part (if any).

- It calls `generate_content` with:
  - the system instruction,
  - the tools,
  - the question + scratchpad + last tool result.

- Output:
  - a `response` object, which may contain:
    - `.text` → next Thought / partial answer
    - `.function_calls` → requested tool actions


In [ ]:
def react_model_step(question, scratchpad, last_tool_part=None):
    contents = [f"User question: {question}\n\nScratchpad:\n{scratchpad}".strip()]

    if last_tool_part is not None:
        contents.append(last_tool_part)

    response = client.models.generate_content(
        model=MODEL_ID,
        contents=contents,
        config=types.GenerateContentConfig(
            tools=[react_tool],
            system_instruction=REACT_SYSTEM_PROMPT,
        ),
    )
    return response

### Step 4 - Tool Dispatcher

We'll add a small helper that takes a `tool_call` from the model and
routes it to the correct Python implementation:

- `get_weather` → `get_weather_impl`
- `calculate` → `calculate_impl`
- `get_country_info` → `get_country_info_impl`


In [ ]:
def run_tool(tool_call):
    name = tool_call.name
    args = tool_call.args or {}

    if name == "get_weather":
        return get_weather_impl(args.get("city", ""))

    if name == "calculate":
        return calculate_impl(args.get("expression", ""))

    if name == "get_country_info":
        return get_country_info_impl(args.get("country", ""))

    return {"error": f"Unknown tool {name}"}

### Step 5 - Full ReAct Loop

We will now combine everything:

- Initialize an empty `scratchpad`.
- For up to `max_steps`:
  1. Call `react_model_step`.
  2. Append the model's `Thought` to the scratchpad.
  3. If there are **no function calls**, treat this as the **final answer**.
  4. If there is a function call, run the tool (Action), get the Observation.
  5. Append the Observation to the scratchpad.
  6. Wrap the tool result as a `function_response` and pass it into the next step.

This implements:

> **Thought → Action (tool) → Observation → Thought → … → Final Answer**

In [ ]:
def run_react_agent(question: str) -> str:
    scratchpad = ""
    last_tool_part = None

    while True:
        # 1. Ask model what to do next
        response = react_model_step(question, scratchpad, last_tool_part)

        # 2. If the model gives a final answer (no tool call), return it
        if not response.function_calls:
            final = (response.text or "").strip()
            print("[Final Answer]:", final)
            return final

        # 3. Model requested a tool → handle the action
        tool_call = response.function_calls[0]
        print(f"[Action]: {tool_call.name} {tool_call.args}")

        tool_result = run_tool(tool_call)
        print("[Observation]:", tool_result)

        # Update internal scratchpad
        scratchpad += f"\nObs: {tool_result}"

        # Prepare the tool response for the next loop turn
        last_tool_part = types.Part.from_function_response(
            name=tool_call.name,
            response=tool_result,
        )

## Step 6 - Try the ReAct Agent

Now we can ask questions that may require:

- one tool (weather),
- another tool (calculator),
- the new tool (country info),
- or a combination of tools + reasoning.

Try prompts like:
- **Weather + reasoning**
  - "What's the temperature in Berlin and how should I dress for that?"
- **Math**
  - "If I have 1200 euros and spend 350, how much is left?"
- **Country info**
  - "Tell me about Pakistan and its capital in 2-3 sentences."
- **Mixed reasoning**
  - "If it's hotter than 25°C in Lahore, suggest a day activity. Otherwise suggest something cozy indoors."
  - "Compare Germany and Pakistan based on their capitals and regions."


In [ ]:
# Example 1: Weather + reasoning
#run_react_agent("What's the temperature in Berlin and how should I dress for that?")

# Example 2: Math
# run_react_agent("If I have 1200 euros and spend 350, how much is left?")

# Example 3: Country info
# run_react_agent("Tell me something about Pakistan and its capital.")

# Example 4: Mixed reasoning across tools
run_react_agent("If it's hotter than 25°C in Lahore, suggest an outdoor activity. Otherwise suggest something indoors. What's the temperature in Berlin and how should I dress for that? ")

[Action]: get_weather {'city': 'Lahore'}
[Observation]: {'city': 'Lahore', 'temperature_c': 32, 'condition': 'hot'}
[Action]: get_weather {'city': 'Berlin'}
[Observation]: {'city': 'Berlin', 'temperature_c': 17, 'condition': 'cloudy'}
[Final Answer]: In Lahore, it's 32°C and hot, so I suggest an outdoor activity like visiting Shalimar Gardens. In Berlin, it's 17°C and cloudy, so I recommend wearing a light jacket or sweater.


"In Lahore, it's 32°C and hot, so I suggest an outdoor activity like visiting Shalimar Gardens. In Berlin, it's 17°C and cloudy, so I recommend wearing a light jacket or sweater."